In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dogs-vs-cats/test1.zip
/kaggle/input/dogs-vs-cats/train.zip
/kaggle/input/dogs-vs-cats/sampleSubmission.csv


In [2]:
from zipfile import ZipFile 
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from matplotlib.image import imread
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from PIL import Image

## Extract images from the Zip files

In [3]:
train_file = '/kaggle/input/dogs-vs-cats/train.zip'
test_file = '/kaggle/input/dogs-vs-cats/test1.zip'

with ZipFile(train_file, 'r') as zip: 
    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!') 
    
with ZipFile(test_file, 'r') as zip: 
    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!') 

Extracting all the files now...
Done!
Extracting all the files now...
Done!


In [4]:
training_images = os.listdir('/kaggle/working/train')

## Loading and resizing the images

In [5]:
Label = []
Features = []

for image in training_images:
    Label.append(image[:3])
    img = Image.open('/kaggle/working/train/'+image)
    new_img = img.resize((100, 100))
    new_img.save('/kaggle/working/train/'+image)
    image = imread('/kaggle/working/train/'+image)
    Features.append(image)

In [6]:
feats = np.array(Features) / 255.0
feats.shape

(25000, 100, 100, 3)

In [7]:
labels = pd.DataFrame(Label)
labels[labels=='dog'] = 0
labels[labels=='cat'] = 1

labs = labels.values.astype(np.uint8)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(feats, labs, test_size=0.2)

## Building the NN model

In [9]:
model = keras.models.Sequential()
model.add(layers.Conv2D(64, (3, 3), strides=(1, 1), padding='valid'
                        , activation='relu', input_shape=(100, 100, 3)))
model.add(layers.MaxPool2D((2, 2)))

model.add(layers.Conv2D(64, 3, activation='relu'))
model.add(layers.MaxPool2D((2, 2)))

model.add(layers.Conv2D(32, 3, activation='relu'))
model.add(layers.MaxPool2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(2))

loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim = keras.optimizers.Adam(lr=0.0001)
metrics = ['accuracy']
model.compile(optimizer=optim, loss=loss, metrics=metrics)

## Training the model

In [10]:
batch_size = 50
epochs = 30

model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

Epoch 1/30
400/400 [==============================] - 244s 608ms/step - loss: 0.6688 - accuracy: 0.5789
Epoch 2/30
400/400 [==============================] - 239s 597ms/step - loss: 0.5768 - accuracy: 0.6971
Epoch 3/30
400/400 [==============================] - 231s 578ms/step - loss: 0.5053 - accuracy: 0.7517
Epoch 4/30
400/400 [==============================] - 224s 561ms/step - loss: 0.4448 - accuracy: 0.7879
Epoch 5/30
400/400 [==============================] - 246s 614ms/step - loss: 0.3980 - accuracy: 0.8163
Epoch 6/30
400/400 [==============================] - 231s 577ms/step - loss: 0.3545 - accuracy: 0.8407
Epoch 7/30
400/400 [==============================] - 239s 597ms/step - loss: 0.3145 - accuracy: 0.8625
Epoch 8/30
400/400 [==============================] - 235s 588ms/step - loss: 0.2830 - accuracy: 0.8785
Epoch 9/30
400/400 [==============================] - 235s 587ms/step - loss: 0.2423 - accuracy: 0.8986
Epoch 10/30
400/400 [==============================] - 237s 593m

## Evaluation

In [11]:
model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

100/100 [==============================] - 13s 123ms/step - loss: 1.1917 - accuracy: 0.8084


[1.191707730293274, 0.8083999752998352]